In [ ]:
#Import necessary modules
import requests
import json
import boto3
from boto3.dynamodb.conditions import Key, Attr
import numpy as np
import pandas as pd
from datetime import datetime

s3 = boto3.client('s3')
dynamodb = boto3.resource('dynamodb')
ddbClient = boto3.client('dynamodb')
inlist = []
outlist = []

def putObjectToS3(data,data2):
    #Write to file
    s3.put_object(Body= "{output:[{Nextpage:" + str(data) + "},"+"{WroteTodbDate:" + str(data2) + "}]}",Bucket= "abtesting-454543535", Key='outputToWebServer' + ".json")

#create a function to sample from beta distribution
def sample(a,b):
#Function to sample from distribution
    return np.random.beta(a, b)

#Read from thompson sampleing scratchpad dynamodb database and return latest row
def readLatestScratchPadTable():
#read from Sratch Pad table
        currentBatchTrail = 0
        NoOfUsersProcessed = 0
        l1Alpha = 0
        l1Beta = 0
        l1N = 0
        l1Rewards = 0
        l1ClickPct = 0
        l2Alpha = 0
        l2Beta = 0
        l2N = 0
        l2Rewards = 0
        l2ClickPct = 0
        nextPageShown = 0
        
        SProwList = []
        SPmaxRow = 0
    
        spTableColNames = ['Batch Trial','No. of users processed','L1 Alpha',
                           'L1 Beta','L1 N','L1 Rewards',
                           'L1 Click Pct','L2 Alpha','L2 Beta',
                           'L2 N','L2 Rewards','L2 Click Pct',
                           'Next Page Shown']
    
        spTableName = 'thompsonSamplingDDB-scratchPad'
        paginator = ddbClient.get_paginator('scan')
        spTableIterator = paginator.paginate(TableName = spTableName)
                
        for spRow in spTableIterator:
            for spRowItem in spRow['Items']:
                
                currentBatchTrail = int(spRowItem[spTableColNames[0]]['S'])
                
                if currentBatchTrail >= SPmaxRow:
                    SPmaxRow = currentBatchTrail
                
                NoOfUsersProcessed = int(spRowItem[spTableColNames[1]]['S'])
                l1Alpha = int(spRowItem[spTableColNames[2]]['S'])
                l1Beta = int(spRowItem[spTableColNames[3]]['S'])
                l1N = int(spRowItem[spTableColNames[4]]['S'])
                l1Rewards = int(spRowItem[spTableColNames[5]]['S'])
                l1ClickPct = float(spRowItem[spTableColNames[6]]['S'])
                l2Alpha = int(spRowItem[spTableColNames[7]]['S'])
                l2Beta = int(spRowItem[spTableColNames[8]]['S'])
                l2N = int(spRowItem[spTableColNames[9]]['S'])
                l2Rewards = int(spRowItem[spTableColNames[10]]['S'])
                l2ClickPct = float(spRowItem[spTableColNames[11]]['S'])
                nextPageShown = int(spRowItem[spTableColNames[12]]['S'])
        
                valList = [currentBatchTrail,NoOfUsersProcessed,l1Alpha,
                           l1Beta,l1N,l1Rewards,
                           l1ClickPct,l2Alpha,l2Beta,
                           l2N,l2Rewards,l2ClickPct,nextPageShown]
            
                SProwList.append(valList)
                
        for row in SProwList:
            if row[0] == SPmaxRow:
                return row

#function to read from click data and return wanted metrics
def thompsomSamplingUserClicksProcessing(inBatchTrial, inl1Alpha, inl1Beta, 
                                         inl1Rewards, inl1N, inl2Alpha, 
                                         inl2Beta, inl2Rewards, inl2N,
                                         cT_l1Click, cT_l1noClick, cT_l1N, 
                                         cT_l2Click, cT_l2noClick, cT_l2N, 
                                         cT_lenDF1):
#Process click data

    currentBatchTrial = inBatchTrial
    currentBatchTrial += 1  
    noOfUsersprocessed = cT_lenDF1
    l1Alpha = inl1Alpha
    l1Beta = inl1Beta
    l1N = 0
    l1Rewards = 0
    l1ClickPct = 0
    l2Alpha = inl2Alpha
    l2Beta = inl2Beta
    l2N = 0
    l2Rewards = 0
    l2ClickPct = 0
    nextPage = 0
    
    l1Alpha = l1Alpha + cT_l1Click
    l1Beta = l1Beta + cT_l1noClick
    l1N = cT_l1N
    l1Rewards = cT_l1Click
    l1ClickPct = l1Rewards / l1N
    if np.isnan(l1ClickPct):
        l1ClickPct = 0

    l2Alpha = l2Alpha + cT_l2Click
    l2Beta = l2Beta + cT_l2noClick
    l2N = cT_l2N
    l2Rewards = cT_l2Click
    l2ClickPct = l2Rewards / l2N
    if np.isnan(l2ClickPct):
        l2ClickPct = 0

    page_with_bigger_sample = np.argmax([sample(l1Alpha, l1Beta), sample(l2Alpha, l2Beta)])
    page_with_bigger_sample += 1
    nextPage = page_with_bigger_sample

    outList = [str(currentBatchTrial), str(noOfUsersprocessed), str(l1Alpha), str(l1Beta), str(l1N), 
               str(l1Rewards), str(l1ClickPct), str(l2Alpha), str(l2Beta), str(l2N), str(l2Rewards),
               str(l2ClickPct), str(nextPage)]

    return outList

#Invoke lambda function
def lambda_handler(event, context):
    
    ArchiveTable = dynamodb.Table("AbtestingDBforClicks-ARCHIVE1")
    ClickTable = dynamodb.Table("AbtestingDBforClicks1")
    
    
    scratchPadlist = []
    pTSclickList = []
    df1 = pd.DataFrame({'unique_id': ['ID'], 'date_time': ['time'], 'landing_page': [0], 'click': [0]})
    
    #read click data dynamodb table
    clickTableName = 'AbtestingDBforClicks1'
    paginator = ddbClient.get_paginator('scan')
    clickTableIterator = paginator.paginate(TableName = clickTableName)
    
    for ctRow in clickTableIterator:
        inlist = ctRow['Items']
    
    #get wanted metrics
    for i in inlist:
        landing_page = int(i['landing_page']['S'])
        unique_id = str(i['unique_id']['S'])
        date_time = str(i['date_time']['S'])
        click = int(i['click']['S'])
        
        #put it all in a dict
        clickTableRowDict = {'unique_id':str(unique_id),'date_time':str(date_time),'landing_page':str(landing_page), 'click':str(click)}
        
        #create a copy of data in Archive table
        ArchiveTable.put_item(Item = clickTableRowDict)
        
        #Delete from click data dynamodb table
        ClickTable.delete_item(Key = {'unique_id':str(unique_id)})
        df2 = pd.DataFrame({'unique_id':[unique_id],'date_time':[date_time],'landing_page':[landing_page], 'click':[click]})
        df1 = pd.concat([df1,df2], ignore_index = True)

    df1.drop(index=df1.index[0], axis=0, inplace=True)
    df1.reset_index(drop=True, inplace=True)
    print(df1)
    
    cT_lenDF1 = len(df1)

    #calculate metrics
    l1_click = df1.loc[(df1['landing_page'] == 1) & (df1['click'] == 1)]['click'].count()
    l1_noclick = df1.loc[(df1['landing_page'] == 1) & (df1['click'] == 0)]['click'].count()
    l1N = l1_click + l1_noclick
    l2_click = df1.loc[(df1['landing_page'] == 2) & (df1['click'] == 1)]['click'].count()
    l2_noclick = df1.loc[(df1['landing_page'] == 2) & (df1['click'] == 0)]['click'].count()
    l2N = l2_click + l2_noclick
    noOfUsers = len(df1)
    
    scratchPadlist = readLatestScratchPadTable()
                           
    inBatchTrial = scratchPadlist[0]
    inl1Alpha = scratchPadlist[2]
    inl1Beta = scratchPadlist[3]
    inl1Rewards = scratchPadlist[5]
    inl1N = scratchPadlist[4]
    inl2Alpha = scratchPadlist[7]
    inl2Beta = scratchPadlist[8]
    inl2Rewards = scratchPadlist[10]
    inl2N = scratchPadlist[9]
    
    cT_l1Click = l1_click
    cT_l1noClick = l1_noclick
    cT_l1N = l1N
    cT_l2Click = l2_click
    cT_l2noClick = l2_noclick
    cT_l2N = l2N
    
    
    pTSclickList = thompsomSamplingUserClicksProcessing(inBatchTrial, inl1Alpha, inl1Beta, 
                                                        inl1Rewards, inl1N, inl2Alpha, 
                                                        inl2Beta, inl2Rewards, inl2N,
                                                        cT_l1Click, cT_l1noClick, cT_l1N, 
                                                        cT_l2Click, cT_l2noClick, cT_l2N, 
                                                        cT_lenDF1)
                                                        
    spTableColNames = ['Batch Trial','No. of users processed','L1 Alpha',
                           'L1 Beta','L1 N','L1 Rewards',
                           'L1 Click Pct','L2 Alpha','L2 Beta',
                           'L2 N','L2 Rewards','L2 Click Pct',
                           'Next Page Shown']
                           
                           
    
    
    

    print(spTableColNames)                           
    print(pTSclickList)

    toSPddbDict = {spTableColNames[i]: pTSclickList[i] for i in range(len(spTableColNames))}
    print(toSPddbDict)
    
    #after processing batch write to thompsomSampling DynamoDB Scratchpad table
    table1 = dynamodb.Table("thompsonSamplingDDB-scratchPad")
    table1.put_item(Item = toSPddbDict)
    
    NPS = toSPddbDict['Next Page Shown']
    currentDateTime = datetime.now().isoformat(timespec='seconds')
    NPS_date = currentDateTime
    #putObjectToS3(NPS,NPS_date)
    
    NPS_out = int(NPS)
    
    #Put next page and date into a dict
    outJsonDict = {"output":[{"Nextpage":NPS_out},{"WroteTodbDate":currentDateTime}]}
    #outJsonDict = {"output":[{"Nextpage":int(1)},{"WroteTodbDate":currentDateTime}]}

    
    print(" ")
    print(outJsonDict)